In [1]:
# Importing Dependencies

# For constructing the ConvNetwork that will determine the Q values of moves? 
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input
import numpy as np

# For storing and sampling observations.
import random
from collections import deque
import tqdm

# Construct environment and choose Breakout from Gym
import gym
env = gym.make('Breakout-v0')

Using TensorFlow backend.
[2017-08-18 22:06:53,217] Making new env: Breakout-v0


In [2]:
# Defining a network that takes in two states of a game. 
# Outputs a Q/Reward value of the input move.

i_shape = (2, ) + env.observation_space.shape

# Model is basically 4 Fully Connected Layers.
model = Sequential()
model.add(Flatten(input_shape=i_shape))
model.add(Dense(20, kernel_initializer='uniform', activation='relu'))
model.add(Dense(18, kernel_initializer='uniform', activation='relu'))
model.add(Dense(10, kernel_initializer='uniform', activation='relu'))
model.add(Dense(env.action_space.n, kernel_initializer='uniform', activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [3]:
#HyperParameters

# This is going to log the actions taken
D = deque()

# Number of time steps to observe, probability of a random move. 
# Weighting the future reward, mini batch size. 
observetime = 500
epsilon = 0.7
gamma = 0.9
mb_size = 50

In [4]:
# Define Prior to Implementing Observation Framework

# Start the game
observation = env.reset()

# Observation is the first element in batch
obs = np.expand_dims(observation, axis=0)
done = False
state = np.stack((obs, obs), axis=1)    

In [5]:
# Implement and Execute Observation Framework on defined number of timesteps

for t in tqdm.tqdm(np.arange(observetime)):
    
# Choose random action if within epsilon
    if np.random.random() <= epsilon:
        action = np.random.randint(0, env.action_space.n)
    else:      
# Choose deterministic action based on Q values. 
        Q = model.predict(state)
        action = np.argmax(Q)
    
    observation_new, reward, done, info = env.step(action)
    #  Reward after performing the corresponding action
    
    #  Record Keeping   
    obs_new = np.expand_dims(observation_new, axis=0)
    state_temp = np.append(np.expand_dims(obs_new, axis=0), state[:, :1, :], axis=1)
    D.append((state, action, reward, state_temp, done))
    state = state_temp
    
    if done:
        # Reset the game if finished up
        env.reset() 
        obs = np.expand_dims(observation, axis=0)
        state = np.stack((obs, obs), axis=1)

100%|██████████| 500/500 [00:01<00:00, 306.76it/s]


In [9]:
# Sample Minibatch and define dimensions.

minibatch = random.sample(list(D), mb_size)

inputs_shape = (mb_size, ) + state.shape[1:]
inputs = np.zeros(inputs_shape)
targets = np.zeros((mb_size, env.action_space.n))

In [253]:
# Framework to learn from executed Observation Framework above

for i in tqdm.tqdm(np.arange(mb_size)):
    
    state = minibatch[i][0]
    action = minibatch[i][1]
    reward = minibatch[i][2]
    state_new = minibatch[i][3]
    done = minibatch[i][4] 
    
    # Bellman equation for the Q Function   
    inputs[i:i+1] = np.expand_dims(state, axis=0)
    targets[i] = model.predict(state)
    Q_sa = model.predict(state_new)
    
    # Choose the targets based on whether the game is still on   
    if done:
        targets[i, action] = reward
    else:
        targets[i, action] = reward + gamma * np.max(Q_sa)
        
    #  Train the network to output the Q Function
    model.train_on_batch(inputs, targets)

100%|██████████| 50/50 [00:08<00:00,  6.57it/s]


In [254]:
# Define the game metadata

observation = env.reset()
obs = np.expand_dims(observation, axis=0)
state = np.stack((obs, obs), axis=1)
done = False
total_reward = 0.0

In [255]:
# Play the game based on the framework learned

while not done:
    env.render()
    Q = model.predict(state)
    action = np.argmax(q)
    
    # Feedback
    observation, reward, done, info = env.step(action)
    obs = np.exapnd_dims(observation, axis=0)
    state = np.append(np.exapnd_dims(obs, axis=1), state[:, :1, :], axis=1)
    total_reward = total_reward + reward

NameError: name 'q' is not defined